In [1]:
import jax
import jax.numpy as jnp
from jax import random
from flax import linen as nn
from flax.training import train_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import optax
import pandas as pd
import pickle
import os
import json

In [2]:
folder = "D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/UNIT_MODULE/Parameters/"
epsilon = 0.2
current_time = '2025_01_24_02_47_02'

params_file_path = f'{folder}AP_{epsilon}/trainable_model_{current_time}.json'

In [3]:
# Función para cargar los parámetros iniciales desde el archivo
def load_initial_params(file_path):
    with open(file_path, 'r') as f:
        loaded_params = json.load(f)
    
    # Convertir los valores de listas a arrays de JAX
    def to_jnp_array(data):
        if isinstance(data, dict):
            return {key: to_jnp_array(value) for key, value in data.items()}
        elif isinstance(data, list):
            return jnp.array(data)
        else:
            return data

    return to_jnp_array(loaded_params)

In [4]:
# Generar datos para el módulo de unidades
def generate_unit_data():
    x_data = []
    y_data = []
    for a in range(10):  # Primer número
        for b in range(10):  # Segundo número
            x_data.append([a, b])  # Entrada
            y_data.append((a + b) % 10)  # Unidades de la suma
    return jnp.array(x_data), jnp.array(y_data)

# Generar datos
x, y = generate_unit_data()

# Convertir etiquetas a one-hot (y convertir a formato denso)
encoder = OneHotEncoder(categories='auto')
y_one_hot = encoder.fit_transform(y.reshape(-1, 1)).toarray()

x_train, y_train = x, y_one_hot
x_val, y_val = x, y_one_hot

# Agregar una dimensión para 'timesteps'
x_train = x_train[:, None, :]  # (batch, timesteps, features)
x_val = x_val[:, None, :]

In [5]:
# Definir el modelo en Flax
class unit_LSTMModel(nn.Module):
    @nn.compact
    def __call__(self, x):
        lstm_1 = nn.LSTMCell(features=16)
        lstm_2 = nn.LSTMCell(features=32)
        lstm_3 = nn.LSTMCell(features=16)
        dense = nn.Dense(10)

        carry1 = lstm_1.initialize_carry(jax.random.PRNGKey(0), (x.shape[0],))
        carry2 = lstm_2.initialize_carry(jax.random.PRNGKey(1), (x.shape[0],))
        carry3 = lstm_3.initialize_carry(jax.random.PRNGKey(2), (x.shape[0],))

        for t in range(x.shape[1]):  # Iterar sobre los pasos temporales
            carry1, x_t = lstm_1(carry1, x[:, t])
            carry2, x_t = lstm_2(carry2, x_t)
            carry3, x_t = lstm_3(carry3, x_t)

        hidden_state = carry3[0]  # Estado oculto tiene forma (batch_size, 32)
        final_output = nn.softmax(dense(hidden_state))
        return final_output

In [6]:
# Inicializar modelo y estado de entrenamiento
def create_train_state(rng, learning_rate):
    model = unit_LSTMModel()
    params = model.init(rng, jnp.ones((1, 1, 2)))["params"]
    tx = optax.sgd(learning_rate)
    return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=tx)

def load_train_state(rng, learning_rate, initial_params):
    model = unit_LSTMModel()
    tx = optax.sgd(learning_rate)
    return train_state.TrainState.create(apply_fn=model.apply, params=initial_params, tx=tx)

# Definir la función de pérdida
def compute_loss(params, x, y):
    logits = unit_LSTMModel().apply({"params": params}, x)
    loss = optax.softmax_cross_entropy(logits, y).mean()
    return loss

# Definir la función de evaluación
@jax.jit
def evaluate(params, x, y):
    logits = unit_LSTMModel().apply({"params": params}, x)
    accuracy = jnp.mean(jnp.argmax(logits, axis=-1) == jnp.argmax(y, axis=-1))
    return accuracy

# Entrenamiento
@jax.jit
def train_step(state, x, y):
    loss_fn = lambda params: compute_loss(params, x, y)
    grads = jax.grad(loss_fn)(state.params)
    state = state.apply_gradients(grads=grads)
    return state

In [12]:
initial_params = load_initial_params(params_file_path)

# Entrenamiento principal
rng = random.PRNGKey(10)
state = load_train_state(rng, learning_rate=0.1, initial_params=initial_params)
#state = create_train_state(rng, learning_rate=0.01)
epochs = 1000000
batch_size = 100

for epoch in range(epochs):
    # Entrenamiento por lotes
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        state = train_step(state, x_batch, y_batch)

    # Evaluación al final de la época
    if (epoch + 1) % 1000 == 0 or epoch == 0:
        accuracy = evaluate(state.params, x_val, y_val)
        print(f"Época {epoch + 1}, Precisión: {accuracy:.4f}")
    if accuracy == 1.0:
        print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
        break

# Evaluación final
final_accuracy = evaluate(state.params, x_val, y_val)
print(f"Precisión final: {final_accuracy:.4f}")

Época 1, Precisión: 0.1000
Época 1000, Precisión: 0.1000
Época 2000, Precisión: 0.1200
Época 3000, Precisión: 0.1100
Época 4000, Precisión: 0.1200
Época 5000, Precisión: 0.1100
Época 6000, Precisión: 0.1100
Época 7000, Precisión: 0.1100
Época 8000, Precisión: 0.1100
Época 9000, Precisión: 0.1200
Época 10000, Precisión: 0.1100
Época 11000, Precisión: 0.1100
Época 12000, Precisión: 0.1000
Época 13000, Precisión: 0.1200
Época 14000, Precisión: 0.1400
Época 15000, Precisión: 0.1300
Época 16000, Precisión: 0.1200
Época 17000, Precisión: 0.1200
Época 18000, Precisión: 0.1200
Época 19000, Precisión: 0.1400
Época 20000, Precisión: 0.1500
Época 21000, Precisión: 0.1700
Época 22000, Precisión: 0.1700
Época 23000, Precisión: 0.1600
Época 24000, Precisión: 0.1600
Época 25000, Precisión: 0.1600
Época 26000, Precisión: 0.1700
Época 27000, Precisión: 0.1900
Época 28000, Precisión: 0.2200
Época 29000, Precisión: 0.2800
Época 30000, Precisión: 0.2700
Época 31000, Precisión: 0.2900
Época 32000, Precisió

KeyboardInterrupt: 

In [10]:
# Paso final: Mostrar predicciones
def get_predictions(state, x, y):
    logits = unit_LSTMModel().apply({"params": state.params}, x)
    predictions = jnp.argmax(logits, axis=-1)  # Predicciones del modelo
    true_labels = jnp.argmax(y, axis=-1)  # Etiquetas reales
    return predictions, true_labels

# Obtener todas las predicciones
preds, true_labels = get_predictions(state, x_val, y_val)

# Crear una tabla con las entradas, etiquetas reales y predicciones
results = []
for i in range(len(x_val)):
    x1 = x_val[i, 0, 0].item()  # Extraer x1 (scalar)
    x2 = x_val[i, 0, 1].item()  # Extraer x2 (scalar)
    y_true = true_labels[i].item()  # Etiqueta real (scalar)
    y_pred = preds[i].item()  # Predicción (scalar)
    results.append({"x1": x1, "x2": x2, "y (real)": y_true, "pred": y_pred})

# Convertir a DataFrame para mostrarlo como tabla
df_results = pd.DataFrame(results)
print(df_results)

    x1  x2  y (real)  pred
0    0   0         0     0
1    0   1         1     0
2    0   2         2     0
3    0   3         3     6
4    0   4         4     0
..  ..  ..       ...   ...
95   9   5         4     8
96   9   6         5     8
97   9   7         6     8
98   9   8         7     8
99   9   9         8     8

[100 rows x 4 columns]


In [29]:
folder = 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/Modules'
os.makedirs(folder, exist_ok=True)

# Guardar el modelo en la carpeta 'Hola'
model_path = os.path.join(folder, "unit_module_JAX.pkl")
with open(model_path, "wb") as f:
    pickle.dump(state.params, f)

print(f"Modelo guardado en '{model_path}'")

Modelo guardado en 'D:/OneDrive - Universidad Complutense de Madrid (UCM)/Doctorado/Curriculum_Learning/JAX_MODULES-Easy_Multidigit_Addition_Decimal/Modules\unit_module_JAX.pkl'
